In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pdb
from scipy import integrate
from scipy import special
import numpy as np
from cmath import *
import matplotlib.pyplot as plt
from IPython.display import display, Math, Latex
from patch_geo_func import x_ep, y_ep
from sys import stdout
import warnings
np.seterr(invalid = 'raise', under = 'ignore', over = 'ignore')

In [ ]:
from assign_attr import *
from repel_system import *

In [ ]:
with open('ORcolor-f0011.bin', 'r') as f:
    OR = np.fromfile(f, 'f8')

In [ ]:
#LR_Pi_file = 'Ny-2-LR_Pi.bin'
#LR_Pi_file = 'cortex_nG2-NxNyRatio2-LR_Pi.bin'
#pos_file = 'server_data/test_3d_pos.bin'
#pos_file = 'server_data/test_low_3d_pos.bin'
LR_Pi_file = 'Ny-2-LR_Pi_newFormat.bin'
#pos_file = 'ss_low_3d_pos.bin'
pos_file = 'V1_pos_2D_lowDensity.bin'
#pos_file = 'ss_pos_file.bin'
#OD_file = 'ss_od_file.bin'
#OP_file = None
OPgrid_file = 'ORcolor-f0011.bin'
#vpos_file = 'vpos.bin'
#uniform_pos_file = 'V1_pos.bin'
uniform_pos_file = 'V1_pos_test.bin'
OP_file = 'OP_file.bin'
OD_file = 'OD_file.bin'
VFxy_file ='VFxy_file.bin'

#mMap = macroMap(LR_Pi_file, uniform_pos_file, posUniform = True, OPgrid_file = OPgrid_file, OD_file = OD_file)
mMap = macroMap(LR_Pi_file, pos_file, posUniform = False)
#mMap = macroMap(LR_Pi_file, pos_file, OR_file, posUniform = False, OD_file = OD_file)
if mMap.pODready:
    assert(np.sum(mMap.ODlabel>0) + np.sum(mMap.ODlabel<0) == mMap.networkSize)
    
#pfile = 'p_repel.bin'
#mMap.save(Parallel_file = pfile)
mMap.save(allpos_file = 'test_pos.bin')

In [ ]:
fig = plt.figure('OP-Pi', dpi=600)
ax = fig.add_subplot(111)
ax.plot(mMap.xx, mMap.yy, ',k')
ax.plot(mMap.xx[mMap.Pi>0], mMap.yy[mMap.Pi>0], 'og', ms = 0.1)
ax.plot(mMap.xx[mMap.Pi==0], mMap.yy[mMap.Pi==0], 'or', ms = 0.1)
pick = np.isnan(mMap.OPgrid)
ax.plot(mMap.xx[pick], mMap.yy[pick], ',r')
ax.plot(mMap.xx[~pick], mMap.yy[~pick], ',g')
pick = np.logical_and(np.isnan(mMap.OPgrid), mMap.Pi>0)
ax.plot(mMap.xx[pick], mMap.yy[pick], 'ob', ms = 0.1)
ax.set_aspect('equal')

In [ ]:
fig = plt.figure('macroMap',dpi=600)
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(222, projection='polar')
ax3 = fig.add_subplot(224, projection='polar')
mMap.pODready = False
mMap.pOPready = False
mMap.plot_map(ax1, ax2, ax3, fig.dpi, pltOD = True, pltVF = False, pltOP = True)

ax1.set_aspect('equal')
ax2.set_thetamin(-90)
ax2.set_thetamax(90)
ax2.set_rmax(2.0)
ax2.set_rmin(0.0)
ax2.grid(False)
ax2.tick_params(labelleft=False, labelright=True,
               labeltop=False, labelbottom=True)

ax3.set_thetamin(-90)
ax3.set_thetamax(90)
ax3.set_rmax(2.0)
ax3.set_rmin(0.0)
ax3.grid(False)
ax3.tick_params(labelleft=False, labelright=True,
               labeltop=False, labelbottom=True)

fig.savefig('sobol_test_low_density_uniform.png')

#mMap.save(OD_file = 'OD_file.bin')
#mMap.save(OP_file = 'OP_file.bin')

In [ ]:
# output V1_feature.bin # all should be floating type
with open('V1_feature.bin', 'wb') as f:
    np.array([2]).astype('u4').tofile(f)
    mMap.ODlabel.astype('f4').tofile(f)
    mMap.op.astype('f4').tofile(f)

In [ ]:
# spread uniformly
fig = plt.figure('pos', dpi = 600)
dx = mMap.x[1] - mMap.x[0]
dy = mMap.y[1] - mMap.y[0]
ax1 = fig.add_subplot(121)
ax1.set_xlim(mMap.x[0]-dx/2, mMap.x[-1]+dx/2)
ax1.set_ylim(mMap.y[0]-dy/2, mMap.y[-1]+dy/2)
ax1.set_aspect('equal')
ax2 = fig.add_subplot(122)
ax2.set_xlim(mMap.x[0]-dx/2, mMap.x[-1]+dx/2)
ax2.set_ylim(mMap.y[0]-dy/2, mMap.y[-1]+dy/2)
ax2.set_aspect('equal')
dt0 = np.power(2.0,-np.arange(5,6)).reshape(1,1)
dt1 = np.power(2.0,-np.arange(6,7)).reshape(1,1)
dt = np.hstack((np.tile(dt0,(30,1)).flatten(), np.tile(dt1,(5,1)).flatten()))
#dt = np.tile(dt0,(15,1))
oldpos, cL, cR, nL, nR = mMap.make_pos_uniform(dt, seed = 17482321, ax1 = ax1, ax2 = ax2)

In [ ]:
fig.savefig(pos_file+'_test.png', dpi = 1200)
mMap.save(pos_file = uniform_pos_file)

In [ ]:
fig = plt.figure('vpos', dpi = 600)
dx = mMap.x[1] - mMap.x[0]
dy = mMap.y[1] - mMap.y[0]
ax1 = fig.add_subplot(121)
ax1.set_xlim(mMap.x[0]-dx/2, mMap.x[-1]+dx/2)
ax1.set_ylim(mMap.y[0]-dy/2, mMap.y[-1]+dy/2)
ax1.set_aspect('equal')
ax2 = fig.add_subplot(122)
ax2.set_xlim(mMap.x[0]-dx/2, mMap.x[-1]+dx/2)
ax2.set_ylim(mMap.y[0]-dy/2, mMap.y[-1]+dy/2)
ax2.set_aspect('equal')
if hasattr(mMap, 'vpos'):
    delattr(mMap, 'vpos')
dt = np.power(2.0,-np.arange(6, 7)).reshape(1,1)
dt = np.tile(dt,(1,25)).flatten()
#dt = np.power(2.0,-np.arange(8, 9)).reshape(1,1)
#dt = np.tile(dt,(1,1)).flatten()
p_scale = 3.0
fT = True # firstTime
tmpL = 'tmpL.bin'
tmpVF_L = 'tmpVF_L.bin'
mMap.spread_pos_VF(dt, tmpVF_L, tmpL, 'L', firstTime = fT, ax = ax1, p_scale = p_scale, b_scale = 2.0)
fT = True # firstTime
tmpR = 'tmpR.bin'
tmpVF_R = 'tmpVF_R.bin'
mMap.spread_pos_VF(dt, tmpVF_R, tmpR, 'R', firstTime = fT, ax = ax2, p_scale = p_scale, b_scale = 2.0)
fig.savefig('vpos_in_cortex.png', dpi = 2000)

In [ ]:
mMap.save(VFpolar_file = 'V1_vpos.bin', pos_file = 'V1_pos.bin', Feature_file  = 'V1_feature.bin')

In [ ]:
fig = plt.figure('vpos', dpi = 600)
dx = mMap.x[1] - mMap.x[0]
dy = mMap.y[1] - mMap.y[0]
ax1 = fig.add_subplot(121)
ax1.set_xlim(mMap.x[0]-dx/2, mMap.x[-1]+dx/2)
ax1.set_ylim(mMap.y[0]-dy/2, mMap.y[-1]+dy/2)
ax1.set_aspect('equal')
ax2 = fig.add_subplot(122)
ax2.set_xlim(mMap.x[0]-dx/2, mMap.x[-1]+dx/2)
ax2.set_ylim(mMap.y[0]-dy/2, mMap.y[-1]+dy/2)
ax2.set_aspect('equal')
LRpick = mMap.ODlabel > 0
ax1.plot(np.vstack((mMap.pos[0,LRpick], mMap.vpos[0,LRpick])), np.vstack((mMap.pos[1,LRpick], mMap.vpos[1,LRpick])),'-,c',lw =0.01)
ax1.plot(mMap.vpos[0,LRpick], mMap.vpos[1,LRpick], ',k')
LRpick = mMap.ODlabel < 0
ax2.plot(np.vstack((mMap.pos[0,LRpick], mMap.vpos[0,LRpick])), np.vstack((mMap.pos[1,LRpick], mMap.vpos[1,LRpick])),'-,c',lw =0.01)
ax2.plot(mMap.vpos[0,LRpick], mMap.vpos[1,LRpick], ',k')
fig.savefig('vpos_in_cortex.png', dpi = 2000)

In [ ]:
networkSize = 32*1024

with open('ss_od_file.bin', 'rb') as f:
    LR = np.fromfile(f, 'i4').reshape(networkSize)

nL = np.sum(LR<0)
nR = np.sum(LR>0)

vpos = np.zeros((2,networkSize),dtype='f4')

with open('tmpVF_R.bin','rb') as f:
    vpos[:,LR<0] = np.fromfile(f).reshape((2,nL)).astype('f4')
    
with open('tmpVF_L.bin','rb') as f:
    vpos[:,LR>0] = np.fromfile(f).reshape((2,nR)).astype('f4')
    
with open('vpos.bin', 'wb') as f:
    vpos.tofile(f)

In [ ]:
with open('vpos.bin', 'wb') as f:
    vpos = np.fromfile(f, 'f8') 
with open ('OD_file.bin') as f:
    OD = np.fromfile(f, 'i4')

print(vpos.size)
print(OD.size)
assert(vpos.size == OD.size*2)
fig = plt.figure('vpos', dpi = 600)
ax1 = fig.add_subplot(121)
ax1.plot(vpos[0,OD<0], vpos[1,OD<0], '.')
ax1.set_aspect('equal')
ax2 = fig.add_subplot(122)
ax2.plot(vpos[0,OD>0], vpos[1,OD>0], '.')
ax2.set_aspect('equal')


In [ ]:
import sys
def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

In [ ]:
per_unit_area = np.sqrt(3)*2
#area = mMap.subgrid[0] * mMap.subgrid[1] * np.sum(mMap.LR == 1)
p_scale = 1.0 # potential extension
#cl = np.sqrt((area/np.sum(mMap.ODlabel>0))/per_unit_area)
cl = 1.0
#print(f'{np.pi*cl*cl*np.sum(mMap.ODlabel>0)} ~= {area}')
b_scale = 2.25 # potential extension
psp = 1.0
bsp = 1.0
pk1, pk2 = 2.0, 1.0
bk1, bk2 = 0.5, 0.25
a_particle = cl*p_scale
a_boundary = cl*b_scale
epsilon = 0.01
n = 100
top = 1
bottom = 0
fig = plt.figure('potential-force', dpi = 150)
ax1 = fig.add_subplot(121)
ax1.set_ylim(bottom, top)
ax2 = fig.add_subplot(122)
ax2.set_ylim(bottom, top)
ax2.set_xlim(0, 7.3*cl)
ax1.set_xlim(0, 7.3*cl)
#'''
potential = L_J_potiential(1*a_particle,1*a_particle*psp,pk1,pk2,cl*p_scale)
potential.plot(ax1,ax2,'r')
#boundary:
potential = L_J_potiential(a_boundary,a_boundary*bsp,bk1,bk2,cl*b_scale)
potential.plot(ax1,ax2,'b')
#'''
fig.savefig('L_J.png', dpi = 150)
plt.close(fig)

In [ ]:
#area = mMap.subgrid[0] * mMap.subgrid[1] * np.sum(mMap.Pi > 0)
cl = 1.0
#cl = np.sqrt((area/np.sum(mMap.ODlabel>0))/per_unit_area)*1.05
p_scale = 1.2 # potential extension
b_scale = 1.2 # potential extension
pk1, pk2 = 1.0, 0.5
bk1, bk2 = 1.0, 0.5
a_particle = cl*p_scale
a_boundary = cl*b_scale
potential = L_J_potiential(a_particle,a_particle,pk1,pk2,cl*p_scale)
potential.plot(ax1,ax2,'m')
#boundary:
potential = L_J_potiential(a_boundary,a_boundary,bk1,bk2,cl*b_scale)
potential.plot(ax1,ax2,'c')
potential.print()

In [ ]:
from class_repel_system import *

In [ ]:
#L_J_potential.init(0.039,0.039,2,1,0.39)
p_potential.init(0.039,0.039,2,1,0.39)

In [ ]:
fig = plt.figure('class potential')
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
#L_J_potential.plot(ax1,ax2,':r')
p_potential.plot(ax1,ax2,':r')
ax1.set_ylim(0,1)
ax2.set_ylim(0,1)

In [ ]:
p_potential.print()
L_J_potential.print()